In [ ]:
import requests
import pandas as pd
import time
from urllib import parse

LOC_STR = "서울"
save_PATH = "seoul"
DATE_STR = "210623"

법정동 ID를 API로 수집

In [ ]:
def get_location_ID(string_Location):
    headers = {'appKey': 'API KEY'}
    url = f"https://apis.openapi.sk.com/safecaster/v1/search/location?locale=kr&searchText={string_Location}"
    res = requests.get(url, headers=headers)
    return res.text

In [ ]:
ret = get_location_ID(LOC_STR)
ret = eval(ret)

df = pd.DataFrame()

for idx, i in enumerate(ret["data"]):
    if "dong" in ret["data"][idx]:
        df = df.append(ret["data"][idx], ignore_index=True)

df = df[df["lDongCd"].str[-5:] != "00000"]

In [ ]:
df.to_csv(f"{save_PATH}_code.csv", encoding="utf-8-sig")

<p> 법정동 ID를 기반으로 실제 COVID-19 안전지수 수집 </p>

In [ ]:
df = pd.read_csv(f"./{save_PATH}_code.csv", encoding="utf-8-sig")
df = df.drop(["Unnamed: 0"], axis=1)
df["lDongCd"] = df["lDongCd"].astype(str)

In [ ]:
def get_Covid_Value(target_date, code):
    headers = {'appKey': 'API KEY'}
    url = f"https://apis.openapi.sk.com/safecaster/v1/search/safetyindex/ldongcd/overall?filterDate={target_date}&ldongCd={code}"
    print(url)
    res = requests.get(url, headers=headers)
    return res.text

In [ ]:
delete_list = []
contactDensityPercentile_df = pd.DataFrame()

for code in df["lDongCd"]:
    contactDensityPercentile_dict = {}
    time.sleep(0.1)
    ret = get_Covid_Value(f"20{DATE_STR}", code)
    ret = eval(ret)
    if ret["code"] == "0000":
        for i in ret["data"]:
            contactDensityPercentile_dict[f"{i['hh']}"] = i["contactDensityPercentile"]
        contactDensityPercentile_df = contactDensityPercentile_df.append(contactDensityPercentile_dict, ignore_index=True)
    else:
        delete_list.append(code)

In [ ]:
df = df[~df["lDongCd"].isin(delete_list)]
df = df.reset_index()

In [ ]:
res_df = pd.concat([df,contactDensityPercentile_df],axis=1)

In [ ]:
res_df.to_csv(f"./{save_PATH}_{DATE_STR}_covid.csv", encoding="utf-8-sig")

지도에 시각화를 하기위한 좌표를 T MAP API로 수집

In [ ]:
def get_location(params):
    encoded = parse.urlencode(params)
    response = requests.get("https://apis.openapi.sk.com/tmap/geo/geocoding", params=encoded)
    return response.text

In [ ]:
loc_df = pd.DataFrame()
for i in res_df.index:
    loc_dict = {}
    params={"version":"1",
           "city_do":res_df.iloc[i]["siDo"],
           "gu_gun":res_df.iloc[i]["siGunGu"],
           "dong":res_df.iloc[i]["dong"],
           "coordType":"WGS84GEO",
           "appKey":"API KEY"}
    time.sleep(0.1)
    print(res_df.iloc[i]["dong"])
    ret = get_location(params)
    ret = eval(ret)
    loc_dict["lat"] = ret["coordinateInfo"]["lat"]
    loc_dict["lon"] = ret["coordinateInfo"]["lon"]

    loc_df = loc_df.append(loc_dict, ignore_index=True)
    


In [ ]:
res_df = pd.concat([res_df,loc_df],axis=1)

In [ ]:
res_df.to_csv(f"./{save_PATH}_{DATE_STR}_complete.csv", encoding="utf-8-sig")

데이터 시각화

In [ ]:
#!pip install folium
#!pip install vincent
import folium
import vincent
import json
import numpy as np

In [ ]:
res_df = pd.read_csv(f"./{save_PATH}_{DATE_STR}_complete.csv")
res_df = res_df.drop(["Unnamed: 0", "index"], axis = 1)
res_df["mean"] = res_df.iloc[:,4:-2].mean(axis=1)

In [ ]:
map = folium.Map(location=["37.546517", "126.986576"], zoom_start=11) # 서울시 내의 무작위 좌표

for idx in res_df.index:
    scatter_points = {
    "x": range(0,24),
    "y": res_df.iloc[idx,4:-3],
    }
    scatter_chart = vincent.Line(scatter_points, iter_idx="x", width=400, height=220)
    scatter_chart.legend(title=res_df.iloc[idx,0])
    scatter_json = scatter_chart.to_json()
    
    if res_df['mean'][idx] >= 70.0:
        folium.Marker([res_df['lat'][idx], res_df['lon'][idx]],
                       popup = folium.Popup(max_width=500).add_child(folium.Vega(scatter_json, width=500, height=250)),
                       icon = folium.Icon(color="black", icon="fas fa-biohazard", prefix="fa")).add_to(map)
    elif res_df['mean'][idx] >= 50.0:
        folium.Marker([res_df['lat'][idx], res_df['lon'][idx]],
                       popup = folium.Popup(max_width=500).add_child(folium.Vega(scatter_json, width=500, height=250)),
                       icon = folium.Icon(color="purple", icon="fas fa-biohazard", prefix="fa")).add_to(map)
    elif res_df['mean'][idx] >= 30.0:
        folium.Marker([res_df['lat'][idx], res_df['lon'][idx]],
                       popup = folium.Popup(max_width=500).add_child(folium.Vega(scatter_json, width=500, height=250)),
                       icon = folium.Icon(color="red", icon="fas fa-biohazard", prefix="fa")).add_to(map)
    elif res_df['mean'][idx] >= 10.0:
        folium.Marker([res_df['lat'][idx], res_df['lon'][idx]],
                       popup = folium.Popup(max_width=500).add_child(folium.Vega(scatter_json, width=500, height=250)),
                       icon = folium.Icon(color="orange", icon="fas fa-biohazard", prefix="fa")).add_to(map)
    else:
        folium.Marker([res_df['lat'][idx], res_df['lon'][idx]],
                       popup = folium.Popup(max_width=500).add_child(folium.Vega(scatter_json, width=500, height=250)),
                       icon = folium.Icon(color="blue", icon="fas fa-biohazard", prefix="fa")).add_to(map)


In [ ]:
map.save(f"{save_PATH}_{DATE_STR}.html")